In [1]:
import pandas as pd
import math
import io
from google.colab import files 
#import statsmodels.api as sm
import scipy
from scipy import stats
#from statsmodels.stats import weightstats as stests
#import numpy as np
import csv
from scipy.stats import ttest_ind

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# read source file
#data = pd.read_csv(io.BytesIO(uploaded['data3k690c.csv']))
data = pd.read_csv('/content/drive/My Drive/data12k.csv')
dataOriginal=data
# drop unnecessory columns can add more
data.drop(['CFG_global_cellid','CFG_Cell.CellName','CFG_cell_name'],axis=1, inplace=True)




# fill nuls
data=data.fillna(0)
#print(data.shape)

############
"""
count_conf =int(input("enter configurations count : ")) # replace count-conf as CountCFG
count_Per =int(input("enter perfoamce count : ")) #replace count-Per as CountPP
count_col = count_conf+count_Per #replace count-col as CountClmns
col_last=count_col+1 #replace col-lasr as LasrClmns
"""
###           ####     data preprocessing part

tempdfCFG=data.filter(like='CFG_').head(0)
tempdfPM=data.filter(like='PM_').head(0)
count_conf=len(tempdfCFG.columns) # number of configs
count_Per=len(tempdfPM.columns) # number of perfomnce param

count_col = count_conf+count_Per #number of all columns for ids start from 0 to numofcol-1
col_last=count_col+1 # number of columns 

print('count columns : ',count_col,'    | CFG columns : ',count_conf,'     | PM Columns : ',  count_Per)


data_PP = data.drop(data.iloc[:, 0:count_conf], axis=1, inplace=False) #create PerfmnsPara only data frame
data_CFG = data.drop(data.iloc[:, count_conf:col_last], axis=1, inplace=False) #create Configuration only dataframe
CFGcolumns=list(data_CFG.columns.values)
PPcolumns=list(data_PP.columns.values)

####optional   check for columns that have high number of settings values - mostly these columns can be ID columns, time columns and some other unimportant data you can delete it using drop function
unicounttot=0
for i in range(count_conf): #selec column of configurations
  tempcount=len(data.iloc[:,i].unique())
  #print(i,tempcount)
  if(tempcount>=20):
    print(data.iloc[:,i] )
    print('unique items in col : ',tempcount)

  unicounttot=unicounttot+tempcount

print('number of unique items  : ',unicounttot)
print(data.shape)


In [ ]:
# check for columns that have standard deviation 0. remove is good cause of it will be problem when calculting z score
std0count=0
dataTest=data
dataclearPP=data
std0columnindex=[]
print('check for drop constant output parameters (remove 0 std error ) : ')
for i in range(count_Per):
  if (len(dataTest.iloc[:,count_conf+i].unique())==1):
    print('unique val : ',dataTest.iloc[:,count_conf+i].unique(),' |     count of unique vals : ',len(dataTest.iloc[:,count_conf+i].unique()))
    std0count=std0count+1
    std0columnindex.append(count_conf+i)
    

data = dataTest.drop(dataTest.columns[std0columnindex], axis=1)
print('std 0 columns count    : ',std0count)
print(data)
## std 0 column drop done



tempdfCFG=data.filter(like='CFG_').head(0)
tempdfPM=data.filter(like='PM_').head(0)
count_conf=len(tempdfCFG.columns)
count_Per=len(tempdfPM.columns)

count_col = count_conf+count_Per #replace count-col as CountClmns
col_last=count_col+1 #replace col-lasr as LasrClmns

print('drop constant outputs, \n','count columns : ',count_col,'   |  CFG columns : ',count_conf,'      |   PM Columns : ',  count_Per)


data_PP = data.drop(data.iloc[:, 0:count_conf], axis=1, inplace=False) #create PerfmnsPara only data frame
data_CFG = data.drop(data.iloc[:, count_conf:col_last], axis=1, inplace=False) #create Configuration only dataframe
CFGcolumns=list(data_CFG.columns.values)
PPcolumns=list(data_PP.columns.values)



In [ ]:

uniCFG=[] # store unique values for selected cfg
tempid=0
tempzp=[]
meanuniqPP=[]
stduniqPP=[]
zuniq=[]
tempz,tempm1,tempm2,index0=0,0,0,0


# create 2d list to get output file
csvoutput=[['Configuration column', 'unique value', 'perfomance Parameter', 'Mean_of_PerfomancePara','Stand.Deviation_of_PerfomancePara','Mean_for_Sample_Perfomance','Size_of_Sample','SQRT_of_Sample_Size','Stand.Deviation_of_Sample_for_Perfo.Param', 'z score','Probability']]


print('start calculations--------------')
for i in range(count_conf): #selec column of configurations
  print('Next CFG column (index/TOTAL): ',i+1,'/',count_conf)
  uniCFG=data_CFG.iloc[:,i].unique() #filter unique values in selected CFG column
  for j in range(len(uniCFG)): #get the number of unique vals in selected column and loop 
    tempDF=data[data.iloc[:,i]==uniCFG[j]] #select the dataframe for unique CFG value of column
    for k in range(count_Per): #Loop number of tiles of perfomance paramers
      tempid=count_conf+k #index for Perfomance Parameter

      Size=len(tempDF.index)

      sqrt=math.sqrt(Size) #square root of size of unique data frame
      popumean=data.iloc[:,tempid].mean() #population mean for PP/PM
      unimean=tempDF.iloc[:,tempid].mean() # mean of seleced PM of sample 
      popuzig=data.iloc[:,tempid].std() #total Population Zigma for PM
      

      tempzig=popuzig/sqrt #

      if(tempzig==0):
        tempz=None
        pvalues=None



      else:
        if (unimean>=popumean):
          tempz = (unimean -popumean)/tempzig
          pvalues = scipy.stats.norm.sf(abs(tempz))*100
        else:
          tempz = (popumean-unimean)/tempzig
          pvalues = scipy.stats.norm.sf(abs(tempz))*100

      if(pvalues!=None):
        pvalues = scipy.stats.norm.sf(abs(tempz))*100
        myDataRow=[CFGcolumns[i],uniCFG[j],PPcolumns[k],popumean,popuzig, unimean,Size,sqrt,tempzig,tempz,pvalues]
        #print(index0,myDataRow)
        index0=index0+1
        csvoutput.append(myDataRow)
        """
      else:
        myDataRow=[CFGcolumns[i],uniCFG[j],PPcolumns[k],popumean,popuzig, unimean,sqrt,tempzig,tempz,pvalues]
      """
     
      

    #to next CFG
     
    tempid=0
print('stop calculations--------------')
#output to a csv file
with open('AllProb3k690c.csv','w',newline='') as myoutput:
  writer=csv.writer(myoutput)
  writer.writerows(csvoutput)